# Feature Engineering



**df1 (customer_transaction_details):**


1.   Drop the three Id columns
2.   Drop the outlier row 189 (transactionAmount = 353)
3. Keep paymentMethodRegistrationFailure, paymentMethodType, paymentMethodProvider, transactionFailed, orderState, transactionAmount


**df2 (customers_df):**


1.   Incorporate valid email address column
2.   Incorpoate a column flagging customer with duplication of email address
3.   Incorporate edomain, consolidating all domain names with frequency below 4 as "Other"
4. Incorporate country field from IPAddress. Consolidate countries with frequency less than 7 as "Other"
5. Keep the three numeric columns
6. Drop Billing, customer phone, device, IPAddress
7. Eliminate rows with duplicate emails from cust. Modelling will be performed on transaction rows, not customer rows.


Join the dataframes on emailaddress and then drop emailaddress. Convert categorical variables to one-hot encoding


This will leave 13 variables, which expands to 36 with one-hot encoding.












In [2]:
import pandas as pd
import numpy as np
import re

# df1 (customer_transaction_details)

In [3]:
df1 = pd.read_csv("customer_transaction_details (1).csv",index_col=0)
df1 = df1.drop(['transactionId','orderId','paymentMethodId'], axis=1)
df1 = df1.drop(189,axis=0)

# df2 (customers_df)

In [5]:
df2 = pd.read_csv("customers_df (1).csv",index_col=0)
regex = '''(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])'''
df2['valid_email'] = df2['customerEmail'].apply(lambda x: int(bool(re.fullmatch(regex,x))))
df2['dup_email'] = df2.duplicated(subset=['customerEmail'],keep=False)
df2['edomain'] = df2['customerEmail'].apply(lambda x: x.split("@")[1])
edomainCounts = df2['edomain'].value_counts().head(3).index
df2['edomain'] = df2['edomain'].apply(lambda x: x if x in edomainCounts else "Other")
ipcountry= pd.read_csv("ipcountry.csv")
df2['country'] = ipcountry['Country']
countryCounts = df2['country'].value_counts().head(6).index
df2['country'] = df2['country'].apply(lambda x: x if x in countryCounts else "Other")
df2 = df2.drop(['customerPhone','customerDevice', 'customerBillingAddress', 'customerIPAddress'], axis = 1)
df2 = df2.drop_duplicates(subset='customerEmail', keep='first', inplace=False)

# Concatenate

In [6]:
fraudData = df1.merge(df2, how='left', on='customerEmail')
y = fraudData['Fraud']
X = fraudData.drop(['customerEmail', 'Fraud'], axis = 1)
payment = pd.get_dummies(X['paymentMethodType'],drop_first=True)
provider = pd.get_dummies(X['paymentMethodProvider'],drop_first=True)
state = pd.get_dummies(X['orderState'],drop_first=True)
domain = pd.get_dummies(X['edomain'],drop_first=True)
country = pd.get_dummies(X['country'],drop_first=True)
X = X.drop(['paymentMethodType','paymentMethodProvider','orderState','edomain','country'],axis=1)
X = pd.concat([X,payment,provider,state,domain,country],axis=1)

# Final balance of data: 41% fraud

In [7]:
np.mean(y)

0.4115755627009646